# 제주도 식당의 리뷰와 정보들을 네이버에서 크롤링해오기
- 제주도식당_전처리.csv를 사용

In [1]:
import pandas as pd
import random
import time

import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

C:\Users\clclc\AppData\Local\Temp\ipykernel_15688\1771813411.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 크롤링에 필요한 데이터 불러오기

In [2]:
df = pd.read_csv("Data/제주도식당_전처리.csv", index_col=False)

In [3]:
df.head()

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,도로명전체주소,lat,lng
0,탑동에이치지,일반음식점,기타,제주특별자치도 제주시 삼도이동 1120-2,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",33.5151128,126.5202001
1,에릭스에스프레소,일반음식점,기타,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",33.5214225,126.8588743
2,일품순두부한림점,일반음식점,한식,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",33.4192601,126.2674304
3,김복남맥주제주아라점,일반음식점,식육(숯불구이),제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",33.4714278,126.5457848
4,봉플라봉뱅,일반음식점,기타,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",33.4879976,126.4978933


### 크롤링에 필요한 컬럼만 뽑아서 저장하기

In [4]:
scrapy_df = df[["사업장명", "도로명전체주소"]]

In [5]:
scrapy_df.head()

,사업장명,도로명전체주소
0,탑동에이치지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)"
1,에릭스에스프레소,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층"
2,일품순두부한림점,"제주특별자치도 제주시 한림읍 한림상로 237, 1층"
3,김복남맥주제주아라점,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)"
4,봉플라봉뱅,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)"


### 네이버 지도창에서 검색할 때 정확도를 높이기 위해 도로명 주소의 , 뒤에는 제거
> 이유는 네이버 지도탭에서 검색시 "사업장명" + "도로명전체주소" 형식으로 검색하면 정확히 원하는 식당이 뜨는 빈도수가 높기 때문에   
도로명전체주소가 필요한 것인데, 문제는 도로명주소에 상세주소가 적혀있으면 검색이 안될 확률이 높았음   
때문에 ','뒤에 있는 상세주소를 제거할 필요가 있음


In [6]:
scrapy_df['도로명전체주소'] = scrapy_df['도로명전체주소'].str.split(',', n=1).str[0]

In [7]:
scrapy_df.head()

,사업장명,도로명전체주소
0,탑동에이치지,제주특별자치도 제주시 무근성안길 16
1,에릭스에스프레소,제주특별자치도 제주시 구좌읍 구좌로 77
2,일품순두부한림점,제주특별자치도 제주시 한림읍 한림상로 237
3,김복남맥주제주아라점,제주특별자치도 제주시 중앙로 581
4,봉플라봉뱅,제주특별자치도 제주시 문송1길 6-1


---
## 크롤링하기

In [8]:
# 크롬 드라이버 실행
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [9]:
# 리뷰를 추출해주는 함수
def extract_review():
    # 리뷰 추출
    rev = []  # 추출한 리뷰 저장
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
    time.sleep(0.5)
    for i in range(1, 31): # 더보기 눌러놓고 30개 가져오기
        try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
            # print('리뷰가 저장됨')
            time.sleep(0.5)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
            # print('리뷰가 저장됨')
            rev.append(comment)
        except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
            try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                time.sleep(0.5)
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                # print('리뷰가 저장됨')
                rev.append(comment)
            except:
                # print('이상한 리뷰임')
                rev.append(pd.NA)

    # print('더 이상 리뷰가 없음')
    return rev

In [10]:
count = 1 # 성공한 크롤링 카운트
failedCount = 1 # 실패한 크롤링 카운트

crawlingReviewData = [] # 식당 리뷰를 저장할 리스트
crawlingImageData = [] # 식당 이미지를 저장할 리스트
crawlingCategoryData = [] # 식당 업종분류를 저장할 리스트
crawlingVisitCountData = [] # 식당 방문자리뷰 수를 저장할 리스트
crawlingBlogCountData = [] # 식당 블로그리뷰 수를 저장할 리스트
crawlingRatingData = [] # 식당 평점을 저장할 리스트
# crawlingTimeData = [] # 식당 운영시간을 저장할 리스트

result_df = pd.DataFrame()
# len(scrapy_df)
for i in range(len(scrapy_df)):
    url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
    # url = f"https://map.naver.com/v5/search/숙성도 노형본관"
    # url = f"https://map.naver.com/v5/search/가시림 카페 제주 서귀포시 표선면 녹산로 5번길 171"
    driver.get(url)
    time.sleep(2)
    #app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a > div:nth-child(2) > div > span.A_cdD > div
    #app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a > div:nth-child(2)
    try:
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 이미지 3장 뽑아서 저장하기
        images = []  # 이미지 링크를 저장할 리스트
        for i in range(1, 4):
            image_element = soup.select_one(f"#ibu_{i}")  # ibu_1 ~ 3 이런식으로 이미지의 id가 갯수별로 형성됨
            if not image_element:
                continue  # 이미지 요소가 없으면 for 루프 종료
            style = image_element["style"]
            background_image = style.split("url(")[1].split(")")[0]
            images.append(background_image)  # 이미지 링크를 리스트에 추가

        # 음식 카테고리 뽑기 (버튼이라서 CSS_SELECTOR를 써야함)
        foodCategory = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.DJJvD').text

        # # 식당 운영시간 뽑기
        # toggleButton = driver.find_element(By.CSS_SELECTOR, ".O8qbU.pSavy a") # 토글버튼 지정
        # toggleButton.click()
        # time.sleep(1)
        # print("dfdsfsd")
        
        # # timeText = soup.select(".A_cdD")
        # # 요일에 대한 정보 추출
        # everyday_info = soup.find('span', class_='i8cJw').get_text(strip=True)

        # # 영업 시간에 대한 정보 추출
        # business_hours_info = soup.find('div', class_='H3ua4').get_text(strip=True)

        # print("매일 정보:", everyday_info)
        # print("영업 시간 정보:", business_hours_info)
        # time.sleep(0.5)
        # print(timeText)
        


        # 리뷰 탭을 선택하기 전에 버튼 갯수 파악하기 (별점 4.7, 방문자리뷰 100, 블로그리뷰 120) 이런식으로 된곳
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        
        if len(lists) > 2: # (별점이 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            # 별점 수, 리뷰 수 긁어오기
            ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text
            
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(1)
        
        else: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            # 리뷰 수 긁어오기
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)

        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
        time.sleep(0.5)
        review = extract_review() # 리뷰 추출 함수 호출
        print(f'현재까지 성공한 크롤링 갯수: {count}')
        crawlingReviewData.append(review)
        crawlingImageData.append(images)
        crawlingCategoryData.append(foodCategory)
        crawlingRatingData.append(ratingText.replace("별점", ""))
        crawlingVisitCountData.append(visitText.split()[-1])
        crawlingBlogCountData.append(blogText.split()[-1])
        count += 1
        ## 별점이 있고 없고에 따라 공백 데이터 추가해주는거 필요
    except:
        crawlingReviewData.append('x')
        crawlingImageData.append('x')
        crawlingCategoryData.append('x')
        crawlingRatingData.append('x')
        crawlingVisitCountData.append("x")
        crawlingBlogCountData.append("x")
        print(f'현재까지 실패한 크롤링 갯수: {failedCount}')
        failedCount += 1


    result_df = pd.DataFrame({'foodCategory': crawlingCategoryData, 'images': crawlingImageData, 'rating': crawlingRatingData, 'visitReviewCount': crawlingVisitCountData, 'blogReviewCount': crawlingBlogCountData,'review': crawlingReviewData})


현재까지 실패한 크롤링 갯수: 1
현재까지 성공한 크롤링 갯수: 1
현재까지 성공한 크롤링 갯수: 2
현재까지 실패한 크롤링 갯수: 2
현재까지 성공한 크롤링 갯수: 3
현재까지 실패한 크롤링 갯수: 3
현재까지 실패한 크롤링 갯수: 4
현재까지 성공한 크롤링 갯수: 4
현재까지 성공한 크롤링 갯수: 5
현재까지 성공한 크롤링 갯수: 6
현재까지 성공한 크롤링 갯수: 7
현재까지 성공한 크롤링 갯수: 8
현재까지 성공한 크롤링 갯수: 9
현재까지 실패한 크롤링 갯수: 5
현재까지 실패한 크롤링 갯수: 6
현재까지 실패한 크롤링 갯수: 7
현재까지 성공한 크롤링 갯수: 10
현재까지 성공한 크롤링 갯수: 11
현재까지 성공한 크롤링 갯수: 12
현재까지 성공한 크롤링 갯수: 13
현재까지 성공한 크롤링 갯수: 14
현재까지 성공한 크롤링 갯수: 15
현재까지 실패한 크롤링 갯수: 8
현재까지 성공한 크롤링 갯수: 16
현재까지 성공한 크롤링 갯수: 17
현재까지 실패한 크롤링 갯수: 9
현재까지 성공한 크롤링 갯수: 18
현재까지 실패한 크롤링 갯수: 10
현재까지 실패한 크롤링 갯수: 11
현재까지 성공한 크롤링 갯수: 19
현재까지 성공한 크롤링 갯수: 20
현재까지 실패한 크롤링 갯수: 12
현재까지 실패한 크롤링 갯수: 13
현재까지 성공한 크롤링 갯수: 21
현재까지 성공한 크롤링 갯수: 22
현재까지 실패한 크롤링 갯수: 14
현재까지 성공한 크롤링 갯수: 23
현재까지 실패한 크롤링 갯수: 15
현재까지 성공한 크롤링 갯수: 24
현재까지 성공한 크롤링 갯수: 25
현재까지 성공한 크롤링 갯수: 26
현재까지 실패한 크롤링 갯수: 16
현재까지 실패한 크롤링 갯수: 17
현재까지 성공한 크롤링 갯수: 27
현재까지 실패한 크롤링 갯수: 18
현재까지 성공한 크롤링 갯수: 28
현재까지 성공한 크롤링 갯수: 29
현재까지 성공한 크롤링 갯수: 30
현재까지 성공한 크롤링 갯수: 31
현재까지 실패한 크롤링 갯수: 19
현재까지 실패한 크롤링 갯수: 2

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.94)
Stacktrace:
	GetHandleVerifier [0x00007FF665FEAD22+56930]
	(No symbol) [0x00007FF665F5F622]
	(No symbol) [0x00007FF665E142E5]
	(No symbol) [0x00007FF665DF1D4C]
	(No symbol) [0x00007FF665E823F7]
	(No symbol) [0x00007FF665E97891]
	(No symbol) [0x00007FF665E7BA43]
	(No symbol) [0x00007FF665E4D438]
	(No symbol) [0x00007FF665E4E4D1]
	GetHandleVerifier [0x00007FF666366AAD+3709933]
	GetHandleVerifier [0x00007FF6663BFFED+4075821]
	GetHandleVerifier [0x00007FF6663B817F+4043455]
	GetHandleVerifier [0x00007FF666089756+706710]
	(No symbol) [0x00007FF665F6B8FF]
	(No symbol) [0x00007FF665F66AE4]
	(No symbol) [0x00007FF665F66C3C]
	(No symbol) [0x00007FF665F568F4]
	BaseThreadInitThunk [0x00007FF9329C7344+20]
	RtlUserThreadStart [0x00007FF9340C26B1+33]


In [ ]:
# 출력 옵션 설정 (컬럼 너비 조정)
# pd.set_option('display.max_colwidth', None) # 길이 제한 해제
# pd.set_option('display.max_colwidth', 50) # 길이 기본 값

In [12]:
result_df.tail()

,foodCategory,images,rating,visitReviewCount,blogReviewCount,review
1632,x,x,x,x,x,x
1633,x,x,x,x,x,x
1634,x,x,x,x,x,x
1635,"육류,고기요리","[""https://search.pstatic.net/common/?autoRotat...",4.05,117,27,[굉장히 푸짐한 양이라서 배부르게 먹고 남은 고기는 포장해서 왔어요. 갈비를 배부르...
1636,x,x,x,x,x,x


In [13]:
result_df.to_csv("Data/제주도식당_테스트크롤링.csv", index=False)